# Problem: Evidence Calculation through Thermodynamic Integration

(Developed by Phil Marshall)

## 1. Background

The reason Evidence computation is hard is because the integral we need to do is over the prior PDF. When characterizing the unnormalized posterior PDF we prefer to use efficient sampling methods that focus on the regions of high likelihood, but to compute the Evidence we need to understand the low likelihood prior volume as well. 

Thermodynamic integration achieves this by "tempering" the posterior PDF, such that we begin by sampling the prior and gradually increase the weight of the likelihood. Consider the Evidence functional $E(\lambda)$:

$\;\;\;\;\;E(\lambda) = \int L(\boldsymbol{\theta})^{\lambda}\,\pi(\boldsymbol{\theta})\,d\boldsymbol{\theta}$

The parameter $\lambda$ takes values between 0 to 1. When $\lambda=0$, the integrand is just the prior PDF $\pi(\boldsymbol{\theta})$; when $\lambda=1$, the integrand is the unnormalized posterior. 

> $\lambda$ can be thought of as an inverse temperature. At low $\lambda$ the system is "hot", and samples drawn from $L(\boldsymbol{\theta})^{\lambda}\,\pi(\boldsymbol{\theta})$ will come from all over the prior volume. At high temperature, the system has "cooled", and any Markov chains sampling from $L(\boldsymbol{\theta})^{\lambda}\,\pi(\boldsymbol{\theta})$ will be restricted to the regions of high likelihood.

We can define a sequence of $\lambda$ values, between 0 and 1, that define a "cooling schedule", and at each step, draw samples from $L(\boldsymbol{\theta})^{\lambda}\,\pi(\boldsymbol{\theta})$ in the usual way.

Consider the following functional:

$\log E = \log E(0) + \int_0^1 \frac{d\log E(\lambda)}{d\lambda}\,d\lambda $

Carrying out the derivative we have:

$\;\;\;\;\;\log E = \log E(0) + \int_0^1 \frac{dE(\lambda)/d\lambda}{E(\lambda)}\,d\lambda $

Substituting the evidence integral for $E$ and taking the derivative inside the integral sign we have:

$\;\;\;\;\;\frac{dE(\lambda)}{d\lambda} = \int \log L \cdot L(\boldsymbol{\theta})^{\lambda}\,\pi(\boldsymbol{\theta})\,d\boldsymbol{\theta}$

With the $1/E(\lambda)$ normalization factor, we recognize this integral as being approximable by a sum over samples drawn from  $L(\boldsymbol{\theta})^{\lambda}\,\pi(\boldsymbol{\theta})/E(\lambda)$, such that

$\;\;\;\;\;\frac{1}{E(\lambda)}\frac{dE(\lambda)}{d\lambda} \approx \frac{1}{N_{\lambda}} \sum_{k=1}^{N_{\lambda}} \log L_k$

and so, noting that $E(0)=1$,

$\;\;\;\;\;\log E \approx \int_0^1 \left[ \frac{1}{N_{\lambda}} \sum_{k=1}^{N_{\lambda}} \log L_k \right] d\lambda $

At each value of $\lambda$ in the cooling schedule, we compute the mean log likelihood over a small ensemble of samples drawn from $L(\boldsymbol{\theta})^{\lambda}\,\pi(\boldsymbol{\theta})$.

## 2. Evidence Calculation: Analytic Gaussian

Consider a model with one parameter, $x$, whose prior PDF $P(x|H)$ is a Gaussian, such that $x \sim \mathcal{N}(x_0, \sigma_0^2)$. Suppose we have an observation $x_{obs}$ of $x$, which we assume to have been drawn from a sampling distribution $P(x_{obs}|x, H)$ which is also Gaussian, such that $x_{obs} \sim \mathcal{N}(x, \sigma_{obs}^2)$.

Derive expressions for the mean $\hat{x}$ and variance $\hat{\sigma}^2$ of the unnormalized posterior PDF, which is also proportional to a Gaussian. (You might like to use equations 6b and 6c in [Sam Roweis' Gaussian Identities notes](http://www.cs.nyu.edu/~roweis/notes/gaussid.pdf), and note that the Normal distribution is symmetrical between $x$ and $x_{obs}$.)

The evidence is also analytic, and given by equation 6d in [Sam Roweis' Gaussian Identities notes](http://www.cs.nyu.edu/~roweis/notes/gaussid.pdf). Write down an expression for $E$ in terms of the constants in the problem, $x_0, \sigma_0, x_{obs}$ and $\sigma_{obs}$.

In this fully analytic system, we can draw both prior and posterior samples, and evaluate all PDFs, very easily, using [`scipy.stats.norm`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html) objects. 

Now, let's compare the analytic evidence with some simple estimators. For the following sets of constants, calculate the evidence using:

  a) Your Analytic Expression;
  
  b) Simple Monte Carlo: draw prior samples, compute the likelihood for each one, and average to estimate $E$;
  
  c) The Inverse Likelihood: draw posterior samples, compute the inverse likelihood for each one, and average to estimate $1 / E$;
  
  d) Thermodynamic Integration; draw posterior samples from $L^\lambda\,\pi$, compute the average log likelihood, and hence tabulate $\frac{d\log E(\lambda)}{d\lambda}$ over $\lambda=0$ to 1, and then integrate to estimate $\log E$.   

#### Scenario 0

$\;\;\;\;\;(x_0, \sigma_0, x_{obs}$, $\sigma_{obs}) = (0.0, 1.0, 0.0, 1.0)$

#### Scenario 1

$\;\;\;\;\;(x_0, \sigma_0, x_{obs}$, $\sigma_{obs}) = (0.0, 1.0, 50.0, 1.0)$

#### Scenario 2

$\;\;\;\;\;(x_0, \sigma_0, x_{obs}$, $\sigma_{obs}) = (0.0, 1.0, 0.0, 0.001)$


What do these scenarios represent in terms of signal to noise of the dataset, and the tension between the prior and the likelihood?


> Pro tips:
* Make a plot of $\frac{d\log E(\lambda)}{d\lambda}$ against $\lambda$ to see what your integrand looks like. 
* In the thermodynamic integration exercise, note that $L^{\lambda} \propto \exp\left[ -\frac{1}{2} \frac{(x - x_{obs})^2}{\sigma_{obs}^2 / \lambda} \right]$, so if you rescale $\sigma_{obs}$ you can still draw samples from the posterior at that $\lambda$ directly!